In [1]:
from pandas import read_csv, to_datetime,Timestamp,DateOffset, DataFrame, read_excel, ExcelWriter
import openpyxl
from datetime import datetime
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
date_du_jour = datetime.today().strftime('%Y-%m-%d')
date_du_jour

'2024-12-24'

In [3]:
file = read_csv("./activity_summary_231224.csv")

In [4]:
lookup = read_excel("./lookup.xlsx")

In [5]:
# Convert 'Last Activity' to naive datetime (without timezone) for consistency
file['Last Activity'] = to_datetime(file['Last Activity'], errors='coerce').dt.tz_localize(None)


# Define the 12-month cutoff using naive datetime for consistency
#cutoff_date = Timestamp.now().tz_localize(None) - DateOffset(months=1)
oct_date_start = "2024-10-01"
oct_date_end = "2024-10-31"

nov_date_start = "2024-11-01"
nov_date_end = "2024-11-30"

dec_cutOff = "2024-12-01"

since_nov= "2024-11-01"

## Month activities

In [6]:
active_users_oct = file[(file['Last Activity'] >= oct_date_start) & (file['Last Activity'] <= oct_date_end)]
num_active_users_oct = len(active_users_oct)

active_users_nov = file[(file['Last Activity'] >= nov_date_start) & (file['Last Activity'] <= nov_date_end)]
num_active_users_nov = len(active_users_nov)


active_users_dec = file[file['Last Activity'] >= dec_cutOff]
num_active_users_dec = len(active_users_dec)

active_users_since_nov = file[file['Last Activity'] >= since_nov]
num_active_users_since_nov = len(active_users_since_nov)

In [7]:
num_active_users_oct
num_active_users_nov 
num_active_users_dec
num_active_users_since_nov

0

20

30

50

In [8]:
active_nov = active_users_nov[['Username','Last Activity']]
active_dec = active_users_dec[['Username','Last Activity']]
active_since_nov = active_users_since_nov[['Username','Last Activity']]

analysis_nov = active_nov.merge(lookup,on="Username",how="left")
analysis_dec = active_dec.merge(lookup,on="Username",how="left")
analysis_since_nov = active_since_nov.merge(lookup,on="Username",how="left")

In [9]:
# Remove "jp.5 -HANWASH" if and only if both "jp.5 -HANWASH" and "Jan.Pooley" exist in the dataframe

# For analysis_dec
if "jp.5 -HANWASH" in analysis_dec["Username"].values and "Jan.Pooley" in analysis_dec["Username"].values:
    analysis_dec = analysis_dec[analysis_dec["Username"] != "jp.5 -HANWASH"]

# For analysis_nov
if "jp.5 -HANWASH" in analysis_nov["Username"].values and "Jan.Pooley" in analysis_nov["Username"].values:
    analysis_nov = analysis_nov[analysis_nov["Username"] != "jp.5 -HANWASH"]

# For analysis_since_nov
if "jp.5 -HANWASH" in analysis_since_nov["Username"].values and "Jan.Pooley" in analysis_since_nov["Username"].values:
    analysis_since_nov = analysis_since_nov[analysis_since_nov["Username"] != "jp.5 -HANWASH"]


In [10]:
usage_analysis_nov = analysis_nov.pivot_table(
    values= "is_duplicated",
    index = "Key_stakeholders",
    aggfunc= "count",
    fill_value=0,
    margins= True,
    margins_name="Total"
)
usage_analysis_nov.rename(
    columns={
        "is_duplicated":"Number of Active Users for the month of November 2024"
    },
    inplace=True
)


In [11]:
usage_analysis_dec = analysis_dec.pivot_table(
    values= "is_duplicated",
    index = "Key_stakeholders",
    aggfunc= "count",
    fill_value=0,
    margins= True,
    margins_name="Total"
)
usage_analysis_dec.rename(
    columns={
        "is_duplicated":"Number of Active Users for the month of December 2024"
    },
    inplace=True
)

In [12]:
usage_analysis_since_nov = analysis_since_nov.pivot_table(
    values= "is_duplicated",
    index = "Key_stakeholders",
    aggfunc= "count",
    fill_value=0,
    margins= True,
    margins_name="Total"
)
usage_analysis_since_nov.rename(
    columns={
        "is_duplicated":"Number of Active Users since 1 November 2024"
    },
    inplace=True
)

In [13]:
# Add a percentage column based on the Total row
usage_analysis_dec["Percentage of Active Users for the month of December 2024"] = (
    usage_analysis_dec["Number of Active Users for the month of December 2024"] 
    / usage_analysis_dec.loc["Total", "Number of Active Users for the month of December 2024"] * 100
).round(0).astype(int).astype(str) + "%"

#usage_analysis_dec.loc["Total", "Percentage of Active Users for the month of December 2024"] = "100%"

usage_analysis_dec

,Number of Active Users for the month of December 2024,Percentage of Active Users for the month of December 2024
Key_stakeholders,,
no,4,14%
yes,25,86%
Total,29,100%


In [14]:
# Add a percentage column based on the Total row
usage_analysis_nov["Percentage of Active Users for the month of November 2024"] = (
    usage_analysis_nov["Number of Active Users for the month of November 2024"] 
    / usage_analysis_nov.loc["Total", "Number of Active Users for the month of November 2024"] * 100
).round(0).astype(int).astype(str) + "%"

#usage_analysis_nov.loc["Total", "Percentage of Active Users for the month of November 2024"] = "100%"

usage_analysis_nov

,Number of Active Users for the month of November 2024,Percentage of Active Users for the month of November 2024
Key_stakeholders,,
no,1,5%
yes,19,95%
Total,20,100%


In [15]:
# Add a percentage column based on the Total row
usage_analysis_since_nov["Percentage of Active Users Since 1 November 2024"] = (
    usage_analysis_since_nov["Number of Active Users since 1 November 2024"] 
    / usage_analysis_since_nov.loc["Total", "Number of Active Users since 1 November 2024"] * 100
).round(0).astype(int).astype(str) + "%"

#usage_analysis_since_nov.loc["Total", "Percentage of Active Users for the month of Since 1 November 2024"] = "100%"

usage_analysis_since_nov

,Number of Active Users since 1 November 2024,Percentage of Active Users Since 1 November 2024
Key_stakeholders,,
no,5,10%
yes,44,90%
Total,49,100%


In [16]:
#usage_analysis
analysis_nov.drop(columns=["is_duplicated"],axis=1, inplace=True)
analysis_dec.drop(columns=["is_duplicated"],axis=1, inplace=True)
analysis_since_nov.drop(columns=["is_duplicated"],axis=1, inplace=True)

In [17]:
from pandas import ExcelWriter

with ExcelWriter(f"./mWater_usage_report_{date_du_jour}.xlsx", engine="openpyxl") as writer:
    usage_analysis_dec.to_excel(writer, sheet_name="Summary", startrow=0, index=True, na_rep="")
    
    next_row = usage_analysis_dec.shape[0] + 5
    usage_analysis_nov.to_excel(writer, sheet_name="Summary", startrow=next_row, index=True, na_rep="")
    
    next_row += usage_analysis_nov.shape[0] + 5
    usage_analysis_since_nov.to_excel(writer, sheet_name="Summary", startrow=next_row, index=True, na_rep="")
    
    analysis_dec.to_excel(writer, sheet_name="List of Users Dec", index=False, na_rep="")
    analysis_nov.to_excel(writer, sheet_name="List of Users Nov", index=False, na_rep="")
    analysis_since_nov.to_excel(writer, sheet_name="List of Users since 1 November", index=False, na_rep="")
